In [118]:
import os
from pathlib import Path


import datacube
import fiona
import geopandas
import rasterio
import rasterio.features

## Create datacube client

In [3]:
config_path = Path(os.getenv("DATACUBE_CONF"))
print(config_path)

/datadisk/dev/sansa-crop-arable-land-fraction/docker/datacube/datacube.conf


In [5]:
dc = datacube.Datacube(
    app="test-calf",
    env="sandbox",
    config=os.getenv("DATACUBE_CONF")
)

/datadisk/miniconda3/envs/sansa-crop-arable-land-fraction/lib/python3.8/site-packages/datacube/drivers/postgres/_connections.py:81: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  EngineUrl(


## Determine ROI

Use fiona to open the input vector file and get its extent

In [9]:
data_dir = Path(os.getenv("DATA_DIR"))
with fiona.open(data_dir / "auxiliary.gpkg", layer="crop-mask") as src:
    roi = src.bounds
print(roi)

(28.1996269226074, -26.9093246459961, 28.2274475097656, -26.9013442993164)


## Load data from the datacube

Since both the spatial and temporal ranges are parametrizable by the user, we need to figure out which products are being queried

In [46]:
base_query = {
    "x": roi[0:3:2],
    "y": roi[1:4:2],
    "time": ("2015-03-01", "2015-05-31"),
    "measurements": [
        "red",
        "nir",
        "spclass"
    ],
    "output_crs": "EPSG:32635",
    "resolution": (-10, 10),
    "resampling": "cubic",
}

In [50]:
input_products = []
for name in dc.list_products().name:
    existing_datasets = dc.find_datasets(
        product=name, 
        x=base_query["x"], 
        y=base_query["y"], 
        time=base_query["time"]
    )
    if len(existing_datasets) > 0:
        input_products.append(name)
print(f"input_products: {input_products}")

input_products: ['test_spot7_gauteng_old_eo3']


In [51]:
ds = dc.load(product=input_products[0], **base_query)

In [93]:
valid_ds = ds.where((ds.spclass != flags["cloud thick"]) & (ds.spclass != flags["cloud thin"]))

In [97]:
ndvi = (valid_ds.nir - valid_ds.red) / (valid_ds.nir + valid_ds.red)

In [98]:
ndvi.sel(time="2015-05-19").values[0][0][0]

0.35095137420718814

In [99]:
ndvi.sel(time="2015-05-26").values[0][0][0]

0.39212513484358147

In [67]:
aggregated_ndvi = ndvi.max(dim="time")

In [105]:
with fiona.open(Path(os.getenv("DATA_DIR")) / "auxiliary.gpkg", layer="crop-mask") as crop_mask_ds:
    print(crop_mask_ds.crs)
    # for feature in crop_mask_ds:
    #    print(feature["geometry"])

{'init': 'epsg:4326'}


In [107]:
dc.list_products().name

id
1    test_spot6_gauteng_old_eo3
2    test_spot7_gauteng_old_eo3
Name: name, dtype: object

In [113]:
type(dc.find_datasets(product="test_spot7_gauteng_old_eo3", time=base_query["time"], x=base_query["x"], y=base_query["y"])[0])

datacube.model.Dataset

In [116]:
with fiona.open(Path(os.getenv("DATA_DIR")) / "auxiliary.gpkg", layer="crop-mask") as crop_mask_ds:
    shapes = [feat["geometry"] for feat in crop_mask_ds]
    crop_mask = rasterio.features.rasterize(((geom, 1) for geom in shapes), out_shape=crop_mask)

<ipython-input-116-2be9bdd2a87a>:2: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  shapes = [feat["geometry"] for feat in crop_mask_ds]


In [117]:
shapes

[{'type': 'Polygon',
  'coordinates': [[(28.20671794454042, -26.904421942546687),
    (28.209438689362916, -26.906442589384753),
    (28.20942884417664, -26.90678838176028),
    (28.209018685896638, -26.90765919694283),
    (28.208533432244884, -26.907196341967662),
    (28.207143920188805, -26.90567376866892),
    (28.206692698068483, -26.905228811082793),
    (28.20671794454042, -26.904421942546687)]]},
 {'type': 'Polygon',
  'coordinates': [[(28.20681511195763, -26.902896992919807),
    (28.20918315679947, -26.904666753902063),
    (28.209386479523307, -26.90476729940975),
    (28.20941428926826, -26.904943738292232),
    (28.209498049481105, -26.905017411219312),
    (28.209465829791093, -26.90588712423559),
    (28.209265675109165, -26.90608411311376),
    (28.209423816959617, -26.906324527270193),
    (28.209440500641826, -26.906430158116603),
    (28.206707021483158, -26.904375844609593),
    (28.206747011588735, -26.903015553416548),
    (28.20681511195763, -26.902896992919807)

In [119]:
crop_mask_gdf = geopandas.read_file(Path(os.getenv("DATA_DIR")) / "auxiliary.gpkg", layer="crop-mask")

/datadisk/miniconda3/envs/sansa-crop-arable-land-fraction/lib/python3.8/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [132]:
type(crop_mask_gdf.iloc[0])

pandas.core.series.Series

In [131]:
crop_mask_gdf.crs.to_epsg()

4326